In [1]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
import time
import re
import pandas as pd
import json


options = webdriver.ChromeOptions()
options.add_argument("headless")

In [2]:
#크롬 브라우저 열기
driver = webdriver.Chrome('C:/chromedriver_win32/chromedriver.exe')
driver.get('https://www.instagram.com/')
p_tag = WebDriverWait(driver, timeout =5).until(EC.presence_of_element_located((By.TAG_NAME, "P")))

time.sleep(3)

#인스타그램 로그인을 위한 계정정보
email='username'
input_id = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[0]
input_id.clear()
input_id.send_keys(email)

password = 'userpw'
input_pw = driver.find_elements_by_css_selector('input._2hvTZ.pexuQ.zyHYP')[1]
input_pw.clear()
input_pw.send_keys(password)
input_pw.submit()

time.sleep(5)

driver.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF').click()
time.sleep(3)
driver.find_element_by_css_selector('button.aOOlW.HoLwm').click()
time.sleep(3)


In [3]:
def clean_text(text):
    cleaned_text = re.sub('[^ ㄱ-ㅣ가-힣]+',' ',text)
    cleaned_text = ' '.join(cleaned_text.split())
    return cleaned_text

In [4]:
def insta_searching(word):
    url ='https://www.instagram.com/explore/tags/' + str(word)
    return url

In [5]:
def click_first(driver):
    first =driver.find_element_by_css_selector('div._9AhH0')
    first.click()
    time.sleep(7)
    
time.sleep(10)

In [21]:
tag_list =[]
comments =[]
comment_more_btn="button.dCJp8.afkep"
comment_ids_objects_css="ul.Mr508 > div.ZyFrc > li.gElp9.rUo9f > div.P9YgZ > div.C7I1f > div.C4VMK > h3" 
comment_texts_objects_css="ul.Mr508 > div.ZyFrc > li.gElp9.rUo9f > div.P9YgZ > div.C7I1f > div.C4VMK > span" 



def get_content(driver):
# 1. 현재 페이지의 HTML 정보 가져오기
    html = driver.page_source
    soup = BeautifulSoup(html, 'lxml')
# 2. 본문내용
    try:
        content = soup.select('div.C4VMK > span')[0].text
    except:
        content = ''
# 3. 본문 내용에서 해시태그 가져오기(정규표현식 활용)
    tags = re.findall('#[A-Za-z0-9가-힣]+', content)
    tag = ''.join(tags).replace("#"," ") # "#" 제거
    tag_data = tag.split()
    for tag_one in tag_data:
        tag_list.append(tag_one)
        
    try:
        while True:
            try:
                more_btn = driver.find_element_by_css_selector(comment_more_btn)
                more_btn.click()
            except:
                break
    except:
        print("----------------------fail to click more btn----------------------------------") 
        #continue

    try:
        comment_data= {}
        comment_ids_objects =driver.find_elements_by_css_selector(comment_ids_objects_css)
        comment_texts_objects = driver.find_elements_by_css_selector(comment_texts_objects_css)
    
        try: 
            for i in range(len(comment_ids_objects)): 
                comment_data[str((i+1))] = {"comment_id":comment_ids_objects[i].text, "comment_text":comment_texts_objects[i].text} 
        except:
            print("fail")
    except:
        comment_id =None
        comment_text = None
        comment_data ={}
        

        
    comment_json = json.dumps(comment_data) 
    comments.append(comment_json)


    data = [content,tag_data, ?] #-->? 자꾸 오류가 떠요..!
    return data

In [22]:
def next_page(driver):
    next_page = driver.find_element_by_css_selector('body > div._2dDPU.CkGkG > div.EfHg9 > div > div > a._65Bje.coreSpriteRightPaginationArrow')
    next_page.click()
    time.sleep(7)

In [23]:
word = "바른먹거리풀무원"
url = insta_searching(word)

driver.implicitly_wait(9)
driver.get(url)
time.sleep(9)

click_first(driver)

result_list=[]

target = 3
for i in range(target):
    try:
        data = get_content(driver)
        result_list.append(data)
        next_page(driver)
    except:
        time.sleep(9)
        next_page(driver)
    

In [24]:
df = pd.DataFrame(result_list)
df.columns = ['본문','해시태그','댓글']
df['본문'] = df['본문'].apply(lambda x : clean_text(x))
df['해쉬태그'] = df['해시태그'].apply(lambda x : clean_text(' '.join(x)))
df['댓글'] = df['댓글'].apply(lambda x : re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\[\]\<\>`\'…》\{\}]','', ''.join(x)))
df = df.drop_duplicates(subset=['본문','해시태그','댓글'])
df

TypeError: unhashable type: 'list'

In [ ]:
#df.to_csv("수정중.csv", header = True, index = False, encoding='utf-8-sig')

In [ ]:
#driver.close()

    try:
        if comment_data != {}:
            
            keys =list(comment_data.keys())
            
            for key in keys: 
                if comment_data[key]['comment_id'] == upload_id: 
                    tags = re.findall('#[A-Za-z0-9가-힣]+', comment_data[key]['comment_text']) 
                    tag = ''.join(tags).replace("#"," ") # "#" 제거 
                    tag_data = tag.split() 
                    for tag_one in tag_data: 
                        instagram_tags.append(tag_one)

    except:
        
        None